## TODO

https://www.kaggle.com/rhtsingh/commonlit-readability-prize-roberta-torch-fit

https://github.com/abhishekkrthakur/commonlit-pairwise-model



## Introduction

Transformers разные слои захватывают разные уровни представлений. Они изучают богатую иерархию лингвистической информации, то есть с поверхностными функциями на нижних уровнях, синтаксическими функциями на средних уровнях и семантическими функциями на более высоких уровнях.

<img src = 'http://jalammar.github.io/images/bert-feature-extraction-contextualized-embeddings.png'/>


Авторы BERT протестировали стратегии встраивания слов, подавая различные комбинации векторов в качестве входных характеристик в BiLSTM, используемый в задаче распознавания именованных сущностей, и наблюдая за полученными баллами F1. Объединение последних четырех слоев дало наилучшие результаты.

Частично это демонстрируется тем, что разные уровни BERT кодируют очень разные виды информации, поэтому соответствующая стратегия объединения будет меняться в зависимости от приложения, поскольку разные уровни кодируют разные виды информации. Это справедливо и для других вариантов.

In [28]:
from IPython.display import clear_output, Image
!pip install transformers
clear_output()

In [19]:
import re
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch import nn, optim
import transformers

path_tr = '/content/drive/MyDrive/CommonLit/input/train.csv'
path_test = '/content/drive/MyDrive/CommonLit/input/test.csv'
path_sub = '/content/drive/MyDrive/CommonLit/input/sample_submission.csv'

SEED =13
np.random.seed(SEED)
torch.manual_seed(SEED)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

tokenizer = transformers.AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path='roberta-base')
model_config = transformers.AutoConfig.from_pretrained(
    pretrained_model_name_or_path='roberta-base',
    num_labels=1)
model = transformers.AutoModel.from_pretrained(
    pretrained_model_name_or_path='roberta-base',
    config=model_config)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
df = pd.read_csv(path_tr)

In [17]:
txt = df.excerpt[:6].values
tkzr = tokenizer.batch_encode_plus(
    list(txt), # batch_text_or_text_pairs has to be a list (got <class 'numpy.ndarray'>)
    truncation=True,
    max_length=256,
    padding='max_length',
    add_special_tokens=True,
    return_attention_mask=True,
    return_token_type_ids=False,
    return_tensors='pt'
)          

In [21]:
out = model(**tkzr)
out

BaseModelOutputWithPoolingAndCrossAttentions([('last_hidden_state',
                                               tensor([[[-0.0676,  0.0801,  0.0089,  ..., -0.0234, -0.0443, -0.0295],
                                                        [-0.0136,  0.2424, -0.1040,  ..., -0.4047,  0.0952, -0.0992],
                                                        [ 0.0357, -0.0299,  0.1018,  ..., -0.2944,  0.0626, -0.1154],
                                                        ...,
                                                        [ 0.0610, -0.0185,  0.1139,  ...,  0.2247,  0.1071,  0.0041],
                                                        [ 0.0610, -0.0185,  0.1139,  ...,  0.2247,  0.1071,  0.0041],
                                                        [ 0.0610, -0.0185,  0.1139,  ...,  0.2247,  0.1071,  0.0041]],
                                               
                                                       [[-0.0829,  0.1356,  0.0139,  ..., -0.0217, -0.0603, -0.049

### pooler_output

In [27]:
out['pooler_output'].shape, out['last_hidden_state'].shape, nn.Linear(768, 1)(out['pooler_output']).shape

(torch.Size([6, 768]), torch.Size([6, 256, 768]), torch.Size([6, 1]))

### last_hidden_state
<img src = 'https://miro.medium.com/max/2120/1*p6PgpOV74U_qLrzr-1_4Zg.png'/>

#### CLS Embeddings